**Task 07: Querying RDF(s)**

In [1]:
!pip install rdflib
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2023-2024/master/Assignment4/course_materials"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 6.1 MB/s eta 0:00:00


First let's read the RDF file

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.namespace_manager.bind('vcard', Namespace("http://www.w3.org/2001/vcard-rdf/3.0#"), override=False)
g.parse(github_storage+"/rdf/example6.rdf", format="xml")

<Graph identifier=N5c3542108ec646b1bd7ceae4aab264cf (<class 'rdflib.graph.Graph'>)>

**TASK 7.1: List all subclasses of "LivingThing" with RDFLib and SPARQL**

In [8]:
from rdflib.plugins.sparql import prepareQuery
ns = Namespace("http://somewhere#")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")

print("RDFLib")
for s,p,o in g.triples((None, RDFS.subClassOf, ns.LivingThing)):
  print(s)

print("SPARQL")
q1 = prepareQuery('''
    SELECT distinct ?subClass
    WHERE {
      ?subClass rdfs:subClassOf ns:LivingThing
    }
    ''',
    initNs = {"rdfs": rdfs, "ns": ns}
)

for r in g.query(q1):
  print(r)

RDFLib
http://somewhere#Person
http://somewhere#Animal
SPARQL
(rdflib.term.URIRef('http://somewhere#Person'),)
(rdflib.term.URIRef('http://somewhere#Animal'),)


**TASK 7.2: List all individuals of "Person" with RDFLib and SPARQL (remember the subClasses)**


In [9]:
rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")

print("RDFLib")
for s,p,o in g.triples((None, RDF.type, ns.Person)):
  print(s)
for s,p,o in g.triples((None, RDFS.subClassOf, ns.Person)):
  for s1,p1,o1 in g.triples((None, RDF.type, s)):
    print(s1)

print("SPARQL")
q2 = prepareQuery('''
    SELECT distinct ?individual
    WHERE {
      {
        ?individual rdf:type ns:Person.
      } UNION {
        ?x rdfs:subClassOf ns:Person.
        ?individual rdf:type ?x
      }
    }
    ''',
    initNs = {"rdfs": rdfs, "rdf": rdf, "ns": ns}
)

for r in g.query(q2):
  print(r)

RDFLib
http://somewhere#SaraJones
http://somewhere#JohnSmith
http://somewhere#JaneSmith
http://somewhere#JimGonzalez
SPARQL
(rdflib.term.URIRef('http://somewhere#SaraJones'),)
(rdflib.term.URIRef('http://somewhere#JohnSmith'),)
(rdflib.term.URIRef('http://somewhere#JaneSmith'),)
(rdflib.term.URIRef('http://somewhere#JimGonzalez'),)


**TASK 7.3: List all individuals of "Person" or "Animal" and all their properties including their class with RDFLib and SPARQL. You do not need to list the individuals of the subclasses of person**


In [11]:
print("RDFLib")
for s, p, o in g.triples((None, RDF.type, ns.Person)):
  for s1, p1, o1 in g.triples((s, None, None)) :
    print (s1, p1, o1)
for s, p, o in g.triples((None, RDF.type, ns.Animal)):
  for s1, p1, o1 in g.triples((s, None, None)) :
    print (s1, p1, o1)

print("SPARQL")
q3 = prepareQuery('''
    SELECT distinct ?individual ?property ?x
    WHERE {
        ?y rdfs:subClassOf ns:LivingThing.
        ?individual rdf:type ?y.
        ?individual ?property ?x
    }
    ''',
    initNs = {"rdfs": rdfs, "rdf": rdf, "ns": ns}
)

for r in g.query(q3):
  print(r)

RDFLib
http://somewhere#SaraJones http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/Given Sara
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/FN Sara Jones
http://somewhere#SaraJones http://www.w3.org/2001/vcard-rdf/3.0/Family Jones
http://somewhere#SaraJones http://xmlns.com/foaf/0.1/knows http://somewhere#RockySmith
http://somewhere#SaraJones http://xmlns.com/foaf/0.1/knows http://somewhere#JaneSmith
http://somewhere#JohnSmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Person
http://somewhere#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0/Given John
http://somewhere#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0/FN John Smith
http://somewhere#JohnSmith http://www.w3.org/2001/vcard-rdf/3.0/Family Smith
http://somewhere#RockySmith http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://somewhere#Animal
http://somewhere#RockySmith http://www.w3.org/2001/vcard-rdf/3

**TASK 7.4:  List the name of the persons who know Rocky**

In [14]:
from rdflib import FOAF
VCARD = Namespace("http://www.w3.org/2001/vcard-rdf/3.0#")

print("RDFLib")
for s, p, o in g.triples((None, FOAF.knows, ns.RockySmith)):
  print(s)

print("SPARQL")
q4 = prepareQuery('''
  SELECT ?individual ?name
  WHERE {
    ?individual FOAF:knows ns:RockySmith.
    ?individual <http://www.w3.org/2001/vcard-rdf/3.0/Given> ?name
  }
  ''',
  initNs = {"FOAF": FOAF, "VCARD": VCARD, "ns": ns}
)

for r in g.query(q4):
  print(r)

RDFLib
http://somewhere#SaraJones
http://somewhere#JaneSmith
SPARQL
(rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.Literal('Sara', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))
(rdflib.term.URIRef('http://somewhere#JaneSmith'), rdflib.term.Literal('Jane', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#string')))


**Task 7.5: List the entities who know at least two other entities in the graph**

In [16]:
print("RDFLib")
lista = []
entities = []
for s, p, o in g.triples((None, FOAF.knows, None)):
  if s in lista:
    entities.append(s)
  lista.append(s)
print(entities)

print("SPARQL")
q5 = prepareQuery('''
  SELECT ?entity
  WHERE {
    ?entity FOAF:knows ?entity2
  }
  GROUP BY ?entity
  HAVING (COUNT(?entity2) >= 2)
  ''',
  initNs = {"FOAF":FOAF}
)

for r in g.query(q5):
  print(r.entity)

RDFLib
[rdflib.term.URIRef('http://somewhere#RockySmith'), rdflib.term.URIRef('http://somewhere#SaraJones'), rdflib.term.URIRef('http://somewhere#JaneSmith')]
SPARQL
http://somewhere#RockySmith
http://somewhere#SaraJones
http://somewhere#JaneSmith
